Polychrom Customizing Simulation

In [18]:
import polychrom
#print(polychrom.__file__)
#check if you have polychrom, if not, please cp to your appropriate directory
#https://github.com/open2c/polychrom.git

In [2]:
#from polychrom import contactmaps
#print(contactmaps.__file__)
#check the path

In [3]:
import os
import sys

In [4]:
import datetime

In [19]:
#print(os.getcwd()) #check your current cwd

In [20]:
import openmm #you mush have openmm to run the simulation

import polychrom
from polychrom import forcekits, forces, simulation, starting_conformations
from polychrom.hdf5_format import HDF5Reporter

In [7]:
def create_polymer(N, polymer_creation_func, method, boxSize=100, r1=10, r2=13, step_size=1.0):

    """
    Polymer Generator
    """
    
    if polymer_creation_func == "grow_cubic":
        polymer = starting_conformations.grow_cubic(N, boxSize, method=method)
            #boxSize : cubic place to grow polymers. size of boxSize**3
            #methods : standard, linear, extended
            #standard : closed chain, ring polymer (ring)
            #linear : open chain, not connected (not a ring)
            #extended : long loop (ring)
            #we do not consider the strand crossing due to Topoisomerase II in this grow_cubic simulation
        return polymer

    elif polymer_creation_func == "create_spiral":
        polymer = starting_conformations.create_spiral(r1, r2, N)
        return polymer

    elif polymer_creation_func == "create_random_walk":
        polymer = starting_conformations.create_random_walk(step_size, N)
        return polymer

    elif polymer_creation_func == "create_constrained_random_walk":
        pass
        #work in progress

    else:
        raise ValueError(f"Unknown polymer_creation_func: {polymer_creation_func}")

In [10]:
def add_confinement(sim, polymer_force_conf, density, k, r="density"):
    if polymer_force_conf == "spherical_confinement":
        sim.add_force(forces.spherical_confinement(sim, density=density, k=k))
            #forces.spherical_confinement(sim, r="density", density=0.85, k=1)
            #~ k is the spring constant, the steepness of the wall (nucleus)
            #r is the radius, the default is density, you can change it manually (unit of nm)

    elif polymer_force_conf == "cylindrical_confinement":
        sim.add_force(forces.cylindrical_confinement(sim, k=k, top=9999))
    #cylindrical_confinement(sim_object, r, bottom=None, k=0.1, top=9999)

    else:
        raise ValueError(f"Unknown polymer_force_conf: {polymer_force_conf} or maybe working in progress, please try another force")

In [11]:
def add_polymer_physics(sim, is_ring = False, trunc = 3.0, angle_k = 1.5, bond_length = 1.0, bond_wiggle = 0.05):
    
    sim.add_force(
        forcekits.polymer_chains(
            sim,
            chains=[(0, None, is_ring)],
            #chains=[(start, end, isRing)]
            #ende == None --> from start to the final one (0 to N-1)
            #isRing == False --> linear, not a ring / isRing == True --> ring
            # By default the library assumes you have one polymer chain
            # If you want to make it a ring, or more than one chain, use self.setChains
            # self.setChains([(0,50,True),(50,None,False)]) will set a 50-monomer ring and a chain from 50 to the end
        
            bond_force_func=forces.harmonic_bonds,
            bond_force_kwargs={
                "bondLength": 1.0,
                "bondWiggleDistance": 0.05,  # Bond distance will fluctuate +- 0.05 on average
            },
            angle_force_func=forces.angle_force,
            angle_force_kwargs={
                "k": 1.5,
                # K is more or less arbitrary, k=4 corresponds to presistence length of 4,
                # k=1.5 is recommended to make polymer realistically flexible; k=8 is very stiff
            },
            nonbonded_force_func=forces.polynomial_repulsive,
            nonbonded_force_kwargs={
                "trunc": trunc,  # this will let chains cross sometimes
                # 'trunc':10.0, # this will resolve chain crossings and will not let chain cross anymore
            },
            except_bonds=True, #if False, then neighbour beads connected to the bond would gather and become so compact... (due to non-bonded forces)
            extra_bonds=None, #or you can put list [(i,j)] of extra bonds
            extra_triplets=None,
            override_checks=False
        )
    )

#making the physics of DNA

In [12]:
def make_simulation(
    N,
    polymer_creation_func,
    method,
    boxSize=100,
    trunc=3.0,
    density=0.85,
    angle_k=1.5,
    is_ring=False,
    k=1.0,
    bond_length=1.0,
    bond_wiggle=0.05,
    polymer_force_conf="spherical_confinement",
    max_data_length = 5
):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_name = f"trajectory_{timestamp}"
    reporter = HDF5Reporter(folder=folder_name, max_data_length = max_data_length, overwrite=True)

    """
    Basic Parameters for the Simulation
    """
    sim = simulation.Simulation(
        platform="CUDA",
        integrator="variableLangevin",  #variablelangevin, langevinmiddle, verlet, variableverlet, brownian
        error_tol=0.003,
        GPU="1",
        collision_rate=0.03,
        N=N,
        save_decimals=2,
        PBCbox=False, #False --> infinite space, like packman effect.
        reporters=[reporter],
        precision="single", #single, mixed, double / but single recommended. double is really slow (10 times slower...)
        max_Ek=10, #raise error if avg kinetic energy exceeds this value ~ <E> > max_Ek --> raises errorr
        temperature=300, #in the unit of Kelvin #default = 300
        verbose=False, #recommend not to change... if True: --> some many stuffs will come out.
        length_scale=1.0, #scaling factor of the system. length_scale of 1.0 --> harmonic bond has the length scale of 1 nm
        mass=100, #unit : amu, sometimes it can be effective according to Raynolds number.
    )

    """
    Polymer Creation
    """

    polymer = create_polymer(N, polymer_creation_func, method, boxSize=100, r1=10, r2=13, step_size=1.0)

    """
    Data Setting ~ load(s) (a) polymer(s)
    """

    sim.set_data(data=polymer, center=True, random_offset=1e-5, report=True)  #it loads a polymer (set particle positions)
    #center = False / True / "zero"
    #True : move center of mass to zero
    #False : ~ True
    #"zero" : then center the data such as all positions are positive and start at zero

    #random_offset : a little noise

    #sim.set_velocities(v)
    #Langevin and Brownian --> it automatically sets the initial velocity. So don't worry.

    """
    Confinement
    """

    add_confinement(sim, polymer_force_conf, density=density, k=k)

    """
    Polymer Physics
    """
    add_polymer_physics(sim, is_ring = is_ring, trunc = trunc, angle_k = angle_k, bond_length = bond_length, bond_wiggle = bond_wiggle)

    assert polymer.shape[0] == N, "Polymer length does not match N"

    return sim, reporter #return the polymer

polymer : N x 3 array of the polymer. (coordinates included) ~ array of positions

In [13]:
#choose the number of monomers
N = 10000
is_ring = False #if False : not a ring, if True : a linear chromosome
trunc = 3.0 #if low, then topoisomerase-like, if high, then no strand crossing

#choose a function for your polymer creation
polymer_creation_func = "grow_cubic" #you can change it manually
#create_spiral(r1, r2, N) : easy mitotic like starting conformation ~ like a sphiral confinement
#create_random_walk(step_size, N) : constrained freely joined chain of length N
#create_constrained_random_walk(N, constraint_f, starting_point, step_size, polar_fixed) : similar to create_random_walk, but with constraint functions
#grow_cubic(N, boxSize, method) : ring / linear polymer on a cubic lattice
method = "standard"
boxSize = 100

#confinement
polymer_force_conf = "spherical_confinement" #making the space!
#spherical_confinement
#cylindrical_confinement
#usually used for nucleus modeling

#spherical_well
#tether_particles
#pull_force
density = 0.85
k = 1.0

#polymer physics
angle_k = 1.5
bond_length = 1.0 #nm
bond_wiggle = 0.05

#max_data_length ~ save every {max_data_length} blocks
max_data_length = 100

In [14]:
sim, reporter = make_simulation(
    N = N,
    polymer_creation_func = polymer_creation_func,
    method = method,
    boxSize = 100,
    trunc = trunc,
    density = density,
    angle_k = angle_k,
    is_ring = is_ring,
    k = k,
    bond_length = bond_length,
    bond_wiggle = bond_wiggle,
    polymer_force_conf = polymer_force_conf,
    max_data_length = max_data_length
)

Exclude neighbouring chain particles from polynomial_repulsive
Number of exceptions: 9999


In [15]:
#number of steps
block_steps = 1000
#timesteps ~ block_timesteps per block
block_timesteps = 100 # ~ 100 integration steps

In [16]:
sim.local_energy_minimization()

INFO:root:Performing local energy minimization
INFO:root:adding force spherical_confinement 0
INFO:root:adding force harmonic_bonds 1
INFO:root:adding force angle 2
INFO:root:adding force polynomial_repulsive 3
INFO:root:Particles loaded. Potential energy is 1.843091
INFO:root:before minimization eK=1.511043768439465, eP=1.843091241440637, time=0.0 ps
INFO:root:Particles loaded. Potential energy is 0.549339
INFO:root:after minimization eK=1.511043768439465, eP=0.3994559562648838, time=0.0 ps


In [17]:
for _ in range(block_steps):  # Do 10 blocks
    sim.do_block(block_timesteps)  # Of 100 timesteps each. Data is saved automatically.
    
sim.print_stats()  # In the end, print very simple statistics
reporter.dump_data()  # always need to run in the end to dump the block cache to the disk

INFO:root:block    0 pos[1]=[-0.5 -1.9 -1.6] dr=0.69 t=7.2ps kin=1.04 pot=1.36 Rg=10.853 SPS=11418 dt=71.8fs dx=16.34pm 
INFO:root:block    1 pos[1]=[-0.3 -2.0 -1.3] dr=0.85 t=14.4ps kin=1.09 pot=1.49 Rg=11.360 SPS=12131 dt=71.8fs dx=16.74pm 
INFO:root:block    2 pos[1]=[0.2 -1.0 -1.8] dr=0.84 t=21.6ps kin=1.11 pot=1.63 Rg=11.776 SPS=12490 dt=71.8fs dx=16.88pm 
INFO:root:block    3 pos[1]=[0.2 -1.3 -1.5] dr=0.80 t=28.8ps kin=1.14 pot=1.74 Rg=11.998 SPS=12182 dt=71.8fs dx=17.12pm 
INFO:root:block    4 pos[1]=[0.0 -1.1 -2.0] dr=0.80 t=36.0ps kin=1.21 pot=1.82 Rg=12.034 SPS=12638 dt=71.8fs dx=17.62pm 
INFO:root:block    5 pos[1]=[1.0 -1.7 -2.1] dr=0.82 t=43.2ps kin=1.27 pot=1.87 Rg=11.940 SPS=12473 dt=71.8fs dx=18.10pm 
INFO:root:block    6 pos[1]=[0.9 -1.5 -2.2] dr=0.85 t=50.3ps kin=1.34 pot=1.88 Rg=11.785 SPS=12126 dt=71.8fs dx=18.61pm 
INFO:root:block    7 pos[1]=[0.1 -0.1 -2.7] dr=0.86 t=57.5ps kin=1.36 pot=1.92 Rg=11.637 SPS=10708 dt=71.8fs dx=18.69pm 
INFO:root:block    8 pos[1]=[0.


 Statistics: number of particles: 10000

Statistics for particle position
     mean position is:  [-0.00695531  0.00026535 -0.01560917]   Rg =  11.694612
     median bond size is  1.0007475828031211
     three shortest/longest (<10)/ bonds are  [0.85884782 0.85917534 0.86391248]    [1.11316676 1.1249872  1.13418871]
     95 percentile of distance to center is:    15.103458515249397
     density of closest 95% monomers is:    0.6582725947354539
     density of the 5% closest to CoM monomers is:    0.7077897008513381
     min/median/mean/max coordinates are: 
     x: -15.69, -0.01, -0.01, 15.73
     y: -16.18, -0.02, 0.00, 15.87
     z: -15.52, 0.03, -0.02, 14.96

Statistics for velocities:
     mean kinetic energy is:  1.4927967306011463 should be: 1.5
     fastest particles are (in kT):  [ 9.15507764  9.39670537  9.46568817  9.68375988 11.42458175]

Statistics for the system:
     Forces are:  ['spherical_confinement', 'harmonic_bonds', 'angle', 'polynomial_repulsive']

Potential Ener

In [ ]:
#let's make Hi-C and do the visualization with Blender!!!
#Yeah~